In [2]:
import numpy as np

class neuralNetwork:
    #ニューラルネットワークの初期化
    def __init__(self,
                input_neurons,
                hidden_neurons,
                output_neurons,
                learning_rate):
        self.inneurons=input_neurons
        self.hneurons=hidden_neurons
        self.oneurons=output_neurons
        self.lr=learning_rate
        #weight_initializeを呼ぶ
        self.weight_initialize()
        
    #重みの初期化
    def weight_initialize(self):
        #入力層と隠れ層の間のリンクの重みの初期値を設定
        #np.random.normal(平均,標準偏差,(行数，列数))
        self.w_h=np.random.normal(0.0,#平均値
                                 pow(self.inneurons,-0.5),#標準偏差
                                  #リンクしているニューロンが多いほど偏差を小さくする
                                  #リンクしているニューロンが多いほどシグモイド関数で1に近い(傾きが0に近い)
                                  #値が発生しやすいため、学習の飽和が起こりやすくなる
                                 (self.hneurons,#隠れ層のニューロン数を行数
                                  self.inneurons+1)#入力層のニューロン数にバイアスを加え、列数とする
                                 )
        #隠れ層と出力層の間のリンクの重みの初期値を設定
        self.w_o=np.random.normal(0.0,
                                 pow(self.hneurons,-0.5),
                                  (self.oneurons,
                                   self.hneurons+1)
                                 )
        
    #活性化関数
    def activation_function(self,x):
        return 1/(1+np.exp(-x))
    
    #ニューラルネットワークの学習を行うメソッド
    def train(self,
             inputs_list,
             targets_list
             ):
        #入力値の配列にバイアス項を追加して１列の行列に変換
        inputs=np.array(np.append(inputs_list,[1]),#配列の末尾にバイアスの値１を追加
                       ndmin=2#2次元化
                       ).T#転置して1列の行列にする
        
        targets=np.array(targets_list,
                        ndmin=2,
                        ).T
        
        #隠れ層への入力信号を計算
        hidden_inputs=np.dot(self.w_h,
                             inputs
                            )
        #活性化関数を適用して隠れ層から出力する
        hidden_outputs=self.activation_function(hidden_inputs)
        #隠れ層の出力行列の末尾にバイアスの値として１を追加
        hidden_outputs=np.append(hidden_outputs,
                                [[1]],#2次元形式でバイアス値を追加
                                axis=0#行を指定（列の場合は１）
                                )
        #出力層への入力信号を計算
        final_inputs=np.dot(self.w_o,
                           hidden_outputs
                           )
        #活性化関数を適用して出力層から出力する
        final_outputs=self.activation_function(final_inputs)
        #目標値と出力層の出力信号の誤差を求める
        output_errors=targets-final_outputs
        #誤差逆伝搬により隠れ層の誤差を求める
        hidden_errors=np.dot(self.w_o.T,
                            output_errors
                            )
        #隠れ層と出力層の間の重みの更新
        self.w_o+=self.lr*np.dot(
            (output_errors*final_outputs*(1.0-final_outputs)),
             np.transpose(hidden_outputs)
        )
        
        #隠れ層の出力エラーからバイアスのエラーを取り除く
        hidden_errors_nobias=np.delete(
            hidden_errors,
            self.hneurons,#バイアスを除くニューロン数をインデックスにする
            axis=0#行の削除を指定
        )
        
        #隠れ層の出力からバイアスを取り除く
        hidden_outputs_nobias=np.delete(
            hidden_outputs,
            self.hneurons,
            axis=0
        )
        
        #入力層と隠れ層の間の重みを更新
        self.w_h+=self.lr*np.dot(
            hidden_errors_nobias*hidden_outputs_nobias*(1.0-hidden_outputs_nobias),
            np.transpose(inputs))
    
    #学習結果を元にテストデータを評価するメソッド
    def evaluate(self,
                inputs_list
                ):
        #テスト用データの配列を１列の行列にする
        inputs=np.array(
            np.append(inputs_list,[1]),
                      ndmin=2
                     ).T
        #隠れ層への入力信号を計算
        hidden_inputs=np.dot(self.w_h,
                             inputs
                            )
        #活性化関数を適用して隠れ層から出力する
        hidden_outputs=self.activation_function(hidden_inputs)
        #出力層への入力信号を計算
        final_inputs=np.dot(self.w_o,
                            np.append(hidden_outputs,[1])
                           )
        #活性化関数を適用して出力層から出力する
        final_outputs=self.activation_function(final_inputs)
        return final_outputs
        
        
input_neurons=3
hidden_neurons=3
output_neurons=3
learning_rate=0.1

n=neuralNetwork(input_neurons,
               hidden_neurons,
               output_neurons,
               learning_rate)

n.train([1.0,1.5,2.0],
        [1.0,1.5,2.0])